# Cox PH model

In [5]:
import numpy as np
import pandas as pd
from datetime import datetime

import pickle

In [6]:
# pd.set_option('display.max_columns', None,'display.max_rows',20)

In [7]:
# importing module
import sys
  
# appending a path
sys.path.append('C:/Users/thseale/Documents/Python Packages')

from lifelines import CoxPHFitter
from lifelines.utils import datetimes_to_durations, k_fold_cross_validation, concordance_index

In [8]:
# importing data
X_tr = pd.read_csv("D:/compiled_data/X_tr.csv").set_index("csid")

y = pd.read_csv("D:/compiled_data/y.csv").set_index("csid")

In [9]:
X_tr

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,mother_still_alive_2,father_still_alive_0,father_still_alive_1,father_still_alive_2,weight_changed_0,weight_changed_1,weight_changed_2,hep_b_0.0,hep_b_1.0,hep_b_2.0
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1799093,0.122746,0.811057,0.592906,1.728535,-0.410683,-0.056127,1.780844,-1.604290,-0.884395,-0.449261,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2022-00229-4744170,-1.250938,-0.168420,-0.422900,0.301515,-0.402491,-0.056127,-0.277026,-1.096393,-1.183793,-0.787761,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2022-00229-1082517,-0.727339,0.040177,0.259928,-0.802161,0.560934,-0.056127,0.472728,0.249563,0.161982,-0.686003,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2022-00229-3397875,1.103472,0.999186,-0.013410,0.773113,1.712047,-0.056127,0.331545,1.448354,1.390868,0.250701,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2022-00229-4372175,1.805687,1.875552,1.842670,1.767708,2.068840,-0.056127,2.404542,1.382637,1.962589,3.259559,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-5565114,0.436560,-0.581341,-0.103120,0.316606,-0.240342,-0.056127,-0.783666,-0.087424,0.314348,0.772009,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2022-00229-4902412,-1.082104,-0.489840,0.057002,-0.483336,-0.998423,-0.056127,0.273152,-1.188568,-1.268748,-0.039717,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2022-00229-4823882,2.495089,0.353427,1.444274,0.410068,1.829385,-0.056127,0.517444,0.972800,1.380486,0.211005,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [10]:
y

,ep_CKB0020_combined_ep,ep_CKB0020_combined_datedeveloped,study_date,subcohort
csid,,,,
2022-00229-1799093,1,2009-07-23T00:00:00Z,2006-04-17T10:08:09Z,False
2022-00229-4744170,1,2011-05-13T00:00:00Z,2006-09-25T15:24:06Z,False
2022-00229-1082517,1,2010-07-14T00:00:00Z,2004-12-16T11:22:31Z,False
2022-00229-3397875,1,2013-06-29T00:00:00Z,2006-07-25T08:54:40Z,False
2022-00229-4372175,1,2013-06-03T00:00:00Z,2007-05-09T09:58:21Z,False
...,...,...,...,...
2022-00229-5565114,0,2018-12-31T00:00:00Z,2006-09-09T09:46:34Z,True
2022-00229-4902412,0,2016-01-02T00:00:00Z,2006-07-24T09:08:24Z,True
2022-00229-4823882,0,2007-09-29T00:00:00Z,2005-04-16T09:56:21Z,True


In [11]:
def date_time_conversion(date):
    return(datetime.strptime(date,"%Y-%m-%dT%H:%M:%SZ"))

In [12]:
# Coverting the times into datetime format
y_tr = y.assign(
    event = y.ep_CKB0020_combined_ep,
    start_time = y.study_date.map(date_time_conversion),
    event_time = y.ep_CKB0020_combined_datedeveloped.map(date_time_conversion)
).drop(columns = y.columns)
y_tr["duration"] = datetimes_to_durations(y_tr["start_time"],y_tr["event_time"])[0]
y_tr["subcohort"] = y["subcohort"]
y_tr = y_tr.drop(columns = ["start_time","event_time"])
y_tr

,event,duration,subcohort
csid,,,
2022-00229-1799093,1,1192.0,False
2022-00229-4744170,1,1690.0,False
2022-00229-1082517,1,2035.0,False
2022-00229-3397875,1,2530.0,False
2022-00229-4372175,1,2216.0,False
...,...,...,...
2022-00229-5565114,0,4495.0,True
2022-00229-4902412,0,3448.0,True
2022-00229-4823882,0,895.0,True


## Fitting Model

### Prentice Cox PH

In [9]:
# creating a case dataframe
cases_prentice_df = X_tr.join(y_tr).query("subcohort == False")

cases_prentice_df = cases_prentice_df.assign(
    # setting events outside subcohort to start just before they occur
    start_time = lambda df: df["duration"] - 0.001,
    # adding appropriate weight
    weight = 1,
    subcohort = False
)

cases_prentice_df

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,weight_changed_1,weight_changed_2,hep_b_0.0,hep_b_1.0,hep_b_2.0,event,duration,subcohort,start_time,weight
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1005462,-1.195952,0.622010,-0.100908,-0.228340,-0.823606,-0.056127,-0.864523,-1.160441,-0.757700,-0.897905,...,0.0,1.0,1.0,0.0,0.0,1,1050.0,False,1049.999,1
2022-00229-1005969,0.409020,0.058957,0.222296,-0.802161,-1.239968,-0.056127,-1.391332,-1.619323,-0.998707,-0.705786,...,1.0,0.0,1.0,0.0,0.0,1,1237.0,False,1236.999,1
2022-00229-1014199,0.054161,0.391132,0.811348,1.243978,-0.781735,-0.056127,0.624358,-0.178877,0.275051,-0.739331,...,0.0,1.0,1.0,0.0,0.0,1,146.0,False,145.999,1
2022-00229-1029504,-1.165206,-1.673662,-2.184277,-0.802161,0.269112,-0.056127,-0.301241,0.691345,0.532407,0.713633,...,0.0,0.0,1.0,0.0,0.0,1,3411.0,False,3410.999,1
2022-00229-1034289,-0.589533,1.080765,-0.237417,-0.802161,-0.743544,-0.056127,0.688579,-0.061868,-0.540294,-0.258872,...,0.0,0.0,1.0,0.0,0.0,1,3382.0,False,3381.999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-6087317,-0.341862,0.351593,0.205567,-0.400126,0.321821,-0.056127,0.878614,0.507184,0.676076,0.060232,...,0.0,0.0,1.0,0.0,0.0,1,2369.0,False,2368.999,1
2022-00229-6088876,0.149734,-0.558810,-0.005420,-0.756520,0.430040,-0.056127,-0.750454,0.826657,1.264328,0.694416,...,0.0,0.0,1.0,0.0,0.0,1,3104.0,False,3103.999,1
2022-00229-6090359,0.124202,-1.010520,-1.265957,-0.802161,-1.332949,-0.056127,-1.672400,-1.188590,-0.531765,-0.936567,...,0.0,0.0,1.0,0.0,0.0,1,768.0,False,767.999,1


In [10]:
# creating a subcohort dataframe
subcohort_prentice_df = X_tr.join(y_tr).query("subcohort == True")

subcohort_prentice_df = subcohort_prentice_df.assign(
        # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
        duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
        # the events start from the origin
        start_time = 0, 
        event = False,
        weight = 1,
        subcohort = True
    )

subcohort_prentice_df

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,weight_changed_1,weight_changed_2,hep_b_0.0,hep_b_1.0,hep_b_2.0,event,duration,subcohort,start_time,weight
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1019291,0.188830,-2.314335,1.867568,-0.802161,2.925413,-0.056127,-1.116515,2.121937,0.366436,1.110488,...,0.0,1.0,1.0,0.0,0.0,False,4734.0,True,0,1
2022-00229-1029425,-1.417434,-0.355111,-0.767257,0.393784,-0.360286,-0.056127,-0.828684,-0.622373,-1.300646,-1.206804,...,0.0,0.0,1.0,0.0,0.0,False,4555.0,True,0,1
2022-00229-1037898,-0.596956,-0.487610,-0.007739,-0.802161,-0.164552,-0.056127,0.819103,-0.407799,0.406625,-0.395019,...,0.0,0.0,1.0,0.0,0.0,False,4503.0,True,0,1
2022-00229-1038110,1.208582,0.117380,1.974078,0.673619,-0.611116,-0.056127,1.383714,-0.322893,-0.073437,-0.011369,...,0.0,0.0,1.0,0.0,0.0,False,4643.0,True,0,1
2022-00229-1043560,-0.896665,-0.075731,0.335512,-0.802161,0.462498,-0.056127,-0.552870,-0.335829,-0.061567,2.274863,...,0.0,0.0,1.0,0.0,0.0,False,4589.0,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-6139477,-0.678425,0.027192,1.262429,0.419242,0.928712,-0.056127,0.473664,0.729747,1.088213,0.140637,...,1.0,0.0,1.0,0.0,0.0,False,5101.0,True,0,1
2022-00229-6140048,-0.640644,1.091666,-1.397685,0.130852,-0.659996,-0.056127,-0.067456,-0.638676,-0.894005,-1.246966,...,1.0,0.0,0.0,1.0,0.0,False,4805.0,True,0,1
2022-00229-6142499,-0.903254,-0.971335,-0.819870,-0.802161,-0.526291,0.872654,-0.114286,-0.513391,-1.040265,-0.591718,...,0.0,0.0,1.0,0.0,0.0,False,4673.0,True,0,1


In [11]:
case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")
case_subcohort_prentice_df

,il8_x100000,tnfrsf9_x100000,tie2_x100000,mcp_3_x100000,cd40_l_x100000,il_1_alpha_x100000,cd244_x100000,egf_x100000,ang_1_x100000,il7_x100000,...,weight_changed_0,weight_changed_1,weight_changed_2,hep_b_0.0,hep_b_1.0,hep_b_2.0,event,duration,start_time,weight
csid,,,,,,,,,,,,,,,,,,,,,
2022-00229-1005462,-1.195952,0.622010,-0.100908,-0.228340,-0.823606,-0.056127,-0.864523,-1.160441,-0.757700,-0.897905,...,0.0,0.0,1.0,1.0,0.0,0.0,1,1050.0,1049.999,1
2022-00229-1005969,0.409020,0.058957,0.222296,-0.802161,-1.239968,-0.056127,-1.391332,-1.619323,-0.998707,-0.705786,...,0.0,1.0,0.0,1.0,0.0,0.0,1,1237.0,1236.999,1
2022-00229-1014199,0.054161,0.391132,0.811348,1.243978,-0.781735,-0.056127,0.624358,-0.178877,0.275051,-0.739331,...,0.0,0.0,1.0,1.0,0.0,0.0,1,146.0,145.999,1
2022-00229-1029504,-1.165206,-1.673662,-2.184277,-0.802161,0.269112,-0.056127,-0.301241,0.691345,0.532407,0.713633,...,1.0,0.0,0.0,1.0,0.0,0.0,1,3411.0,3410.999,1
2022-00229-1034289,-0.589533,1.080765,-0.237417,-0.802161,-0.743544,-0.056127,0.688579,-0.061868,-0.540294,-0.258872,...,1.0,0.0,0.0,1.0,0.0,0.0,1,3382.0,3381.999,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-00229-6139477,-0.678425,0.027192,1.262429,0.419242,0.928712,-0.056127,0.473664,0.729747,1.088213,0.140637,...,0.0,1.0,0.0,1.0,0.0,0.0,0,5101.0,0.000,1
2022-00229-6140048,-0.640644,1.091666,-1.397685,0.130852,-0.659996,-0.056127,-0.067456,-0.638676,-0.894005,-1.246966,...,0.0,1.0,0.0,0.0,1.0,0.0,0,4805.0,0.000,1
2022-00229-6142499,-0.903254,-0.971335,-0.819870,-0.802161,-0.526291,0.872654,-0.114286,-0.513391,-1.040265,-0.591718,...,1.0,0.0,0.0,1.0,0.0,0.0,0,4673.0,0.000,1


In [12]:
cph2 = CoxPHFitter()
cph2.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low variance means that the column cancer_site_1.0 completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [14]:
from sklearn.model_selection import KFold

In [13]:
cox_prentice_penalizer_scores = pd.DataFrame({
    'penalizer' : [],
    'l1_ratio': [],
    'score': []
})
cox_prentice_penalizer_scores

,penalizer,l1_ratio,score


In [15]:
for i in range(6,11):
    for j in range(0,3):
        
        print('penalizer =', i/5, 'l1_ratio =', j/3)
    
        kf = KFold(n_splits = 15)
        kf.get_n_splits(X_tr)

        scores = []

        for train_index, test_index in kf.split(X_tr):
            X_tr_train, X_tr_test = X_tr.iloc[train_index], X_tr.iloc[test_index]
            y_tr_train, y_tr_test = y_tr.iloc[train_index], y_tr.iloc[test_index]

            # creating a case dataframe
            # creating a case dataframe
            cases_prentice_df = X_tr_train.join(y_tr_train).query("subcohort == False")

            cases_prentice_df = cases_prentice_df.assign(
                # setting events outside subcohort to start just before they occur
                start_time = lambda df: df["duration"] - 0.001,
                # adding appropriate weight
                weight = 1,
                subcohort = False
            )

            # creating a subcohort dataframe
            subcohort_prentice_df = X_tr_train.join(y_tr_train).query("subcohort == True")

            subcohort_prentice_df = subcohort_prentice_df.assign(
                    # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
                    duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
                    # the events start from the origin
                    start_time = 0, 
                    event = False,
                    weight = 1,
                    subcohort = True
                )
            
            case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")

            cph = CoxPHFitter(penalizer = i/5, l1_ratio = j/3)
            try:
                cph.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

                event_times = y_tr_test["duration"]
                event_observed = y_tr_test["event"]

                test_preds = cph.predict_partial_hazard(X_tr_test)
                scores.append(concordance_index(event_times, -1*test_preds, event_observed))
            except:
                pass
            
        cox_prentice_penalizer_scores = cox_prentice_penalizer_scores.append({
            'penalizer' : i/5,
            'l1_ratio': j/3,
            'score': np.mean(scores)
        }, ignore_index = True)

penalizer = 1.2 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.2 l1_ratio = 0.3333333333333333


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 1.2 l1_ratio = 0.6666666666666666


...

C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 2.0 l1_ratio = 0.3333333333333333


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

penalizer = 2.0 l1_ratio = 0.6666666666666666


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

In [16]:
cox_prentice_penalizer_scores

,penalizer,l1_ratio,score
0,1.2,0.000000,0.661392
1,1.2,0.333333,0.601144
2,1.2,0.666667,0.596232
3,1.4,0.000000,0.659837
4,1.4,0.333333,0.602153
5,1.4,0.666667,0.596132
6,1.6,0.000000,0.659403
7,1.6,0.333333,0.601963
8,1.6,0.666667,0.596102
9,1.8,0.000000,0.655304


Results seem to be better with no L1 penalty. Let us therefore search the area around 0.5-1.5 to find a suitable penalisation hyperperameter, since it seems that concordance decreases from 1.2

In [ ]:
for i in range(4,5):
    for j in range(0,1):
        
        print('penalizer =', i/10, 'l1_ratio =', j/3)
    
        kf = KFold(n_splits = 15)
        kf.get_n_splits(X_tr)

        scores = []

        for train_index, test_index in kf.split(X_tr):
            X_tr_train, X_tr_test = X_tr.iloc[train_index], X_tr.iloc[test_index]
            y_tr_train, y_tr_test = y_tr.iloc[train_index], y_tr.iloc[test_index]

            # creating a case dataframe
            # creating a case dataframe
            cases_prentice_df = X_tr_train.join(y_tr_train).query("subcohort == False")

            cases_prentice_df = cases_prentice_df.assign(
                # setting events outside subcohort to start just before they occur
                start_time = lambda df: df["duration"] - 0.001,
                # adding appropriate weight
                weight = 1,
                subcohort = False
            )

            # creating a subcohort dataframe
            subcohort_prentice_df = X_tr_train.join(y_tr_train).query("subcohort == True")

            subcohort_prentice_df = subcohort_prentice_df.assign(
                    # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
                    duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
                    # the events start from the origin
                    start_time = 0, 
                    event = False,
                    weight = 1,
                    subcohort = True
                )
            
            case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")

            cph = CoxPHFitter(penalizer = i/10, l1_ratio = j/3)
            try:
                cph.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

                event_times = y_tr_test["duration"]
                event_observed = y_tr_test["event"]

                test_preds = cph.predict_partial_hazard(X_tr_test)
                scores.append(concordance_index(event_times, -1*test_preds, event_observed))
            except:
                pass
            
        cox_prentice_penalizer_scores = cox_prentice_penalizer_scores.append({
            'penalizer' : i/10,
            'l1_ratio': j/3,
            'score': np.mean(scores)
        }, ignore_index = True)

penalizer = 0.4 l1_ratio = 0.0


C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1103: ConvergenceWarning: Column(s) ['rheum_heart_dis_diag'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:801: RuntimeWarning: invalid value encountered in true_divide
  return (X - mean) / std
C:/Users/thseale/Documents/Python Packages\lifelines\utils\__init__.py:1123: ConvergenceWarning: Column cancer_site_1.0 have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['event'].astype(bool)
>>> print(df.loc[events, 'cancer_site_1.0'].var())
>>> print(df.loc[~events, 'cancer_site_1.0'].var())

A very low var

In [ ]:
cox_prentice_penalizer_scores.query("l1_ratio == 0").sort_values("penalizer")

Fitting final model:

In [24]:
pca = PCA(n_components = 0.3)
X_tr_pca = pd.DataFrame(pca.fit_transform(X_tr))
X_tr_pca.index = X_tr.index

# creating a case dataframe
cases_prentice_df = X_tr_pca.join(y_tr).query("subcohort == False")

cases_prentice_df = cases_prentice_df.assign(
    # setting events outside subcohort to start just before they occur
    start_time = lambda df: df["duration"] - 0.001,
    # adding appropriate weight
    weight = 1,
    subcohort = False
)

# creating a subcohort dataframe
subcohort_prentice_df = X_tr_pca.join(y_tr).query("subcohort == True")


subcohort_prentice_df = subcohort_prentice_df.assign(
        # if it is a case, the weight should be the same as the subcohort until close to the time of the event. 
        duration = lambda df: np.where(df["event"], df["duration"] - 0.001, df["duration"]), 
        # the events start from the origin
        start_time = 0, 
        event = False,
        weight = 1,
        subcohort = True
    )

case_subcohort_prentice_df = pd.concat([cases_prentice_df,subcohort_prentice_df]).drop(columns = "subcohort")

cph2 = CoxPHFitter()
cph2.fit(case_subcohort_prentice_df, entry_col = "start_time", duration_col = "duration",event_col = "event",weights_col = "weight",robust = True)

<lifelines.CoxPHFitter: fitted with 1190 total observations, 595 right-censored observations>

In [25]:
pickle.dump(cph2, open('prentice_cox_pca.pkl', 'wb'))